In [2]:
import pandas as pd

In [3]:
import numpy as np

In [4]:
df = pd.read_csv("train_part_1.csv.gz")

In [5]:
train2 = pd.read_csv("train_part_2.csv.gz")

In [6]:
train = pd.concat((df,train2))

In [7]:
train_labels = train['label']

In [8]:
from catboost import CatBoostClassifier, Pool

In [9]:
cat_features = train[['FOI_hits_X','FOI_hits_Y','FOI_hits_Z','FOI_hits_DX','FOI_hits_DY','FOI_hits_DZ','FOI_hits_T','FOI_hits_DT','FOI_hits_S']]

In [10]:
train_without_labels = train.drop('label',axis=1)

In [11]:
train_without_labels = train_without_labels.drop(['FOI_hits_X','FOI_hits_Y','FOI_hits_Z','FOI_hits_DX','FOI_hits_DY','FOI_hits_DZ','FOI_hits_T','FOI_hits_DT','FOI_hits_S'],axis=1)

In [12]:
train_mapped_as_test = train_without_labels.drop(['kinWeight', 'particle_type', 'sWeight', 'weight'],axis=1)

In [13]:
from catboost import FeaturesData

In [14]:
import numpy as np

In [15]:
train_mapped_as_test = train_mapped_as_test.values

In [16]:
train_mapped_as_test = train_mapped_as_test.astype(np.float32)

In [17]:
cat_features = cat_features.values

In [18]:
train_data = Pool(data=FeaturesData((train_mapped_as_test),cat_features),label=train_labels)

In [22]:
model = CatBoostClassifier(iterations = 100, 
                           l2_leaf_reg = 9,
                           boosting_type = 'Plain',
                           bootstrap_type = 'Bernoulli',
                           rsm = 0.1,
                           learning_rate = 0.4,
                           depth = 8,
                           one_hot_max_size = 50,
                           od_type='IncToDec',
                           loss_function = 'Logloss')

In [23]:
model.get_params()

{'boosting_type': 'Plain',
 'bootstrap_type': 'Bernoulli',
 'depth': 8,
 'iterations': 100,
 'l2_leaf_reg': 9,
 'learning_rate': 0.4,
 'loss_function': 'Logloss',
 'od_type': 'IncToDec',
 'one_hot_max_size': 50,
 'rsm': 0.1}

In [ ]:
model.fit(train_data)

0:	learn: 0.3386930	total: 8.62s	remaining: 14m 13s
1:	learn: 0.2479271	total: 14.1s	remaining: 11m 33s
2:	learn: 0.2220790	total: 19.7s	remaining: 10m 36s
3:	learn: 0.2133395	total: 23.9s	remaining: 9m 34s
4:	learn: 0.2080914	total: 28.3s	remaining: 8m 57s
5:	learn: 0.2036821	total: 36.1s	remaining: 9m 25s
6:	learn: 0.2022060	total: 40.1s	remaining: 8m 52s
7:	learn: 0.1992160	total: 50.3s	remaining: 9m 38s
8:	learn: 0.1962520	total: 1m 2s	remaining: 10m 29s
9:	learn: 0.1954550	total: 1m 9s	remaining: 10m 24s
10:	learn: 0.1938385	total: 1m 17s	remaining: 10m 28s
11:	learn: 0.1931372	total: 1m 24s	remaining: 10m 19s
12:	learn: 0.1925665	total: 1m 31s	remaining: 10m 9s
13:	learn: 0.1922515	total: 1m 38s	remaining: 10m 7s
14:	learn: 0.1918210	total: 1m 46s	remaining: 10m 5s
15:	learn: 0.1913988	total: 1m 53s	remaining: 9m 57s
16:	learn: 0.1911016	total: 2m 1s	remaining: 9m 52s
17:	learn: 0.1907162	total: 2m 7s	remaining: 9m 41s
18:	learn: 0.1902670	total: 2m 15s	remaining: 9m 37s
19:	lear

In [ ]:
test_data = pd.read_csv('test_public.csv.gz')

In [24]:
columns_matched = train_without_labels.columns.difference(test_data.columns)

In [25]:
columns_matched

Index(['kinWeight', 'particle_type', 'sWeight', 'weight'], dtype='object')

In [22]:
columns_umatched.shape

(70,)

In [30]:
test_data = test_data.drop(['FOI_hits_X','FOI_hits_Y','FOI_hits_Z','FOI_hits_DX','FOI_hits_DY','FOI_hits_DZ','FOI_hits_T','FOI_hits_DT','FOI_hits_S'],axis=1)

In [31]:
test_data = test_data.values

In [32]:
test_data = test_data.astype(np.float32)

In [33]:
testi = pd.read_csv('test_public.csv.gz')

In [34]:
cat_test_features = testi[['FOI_hits_X','FOI_hits_Y','FOI_hits_Z','FOI_hits_DX','FOI_hits_DY','FOI_hits_DZ','FOI_hits_T','FOI_hits_DT','FOI_hits_S']]

In [35]:
cat_test_features = cat_test_features.values

In [36]:
testing = FeaturesData(test_data,cat_test_features)

In [37]:
pred_class = model.predict(testing)

In [44]:
print(pred_class)

[1. 1. 1. ... 1. 1. 1.]


In [47]:
test_A = pd.read_csv('test_public.csv.gz')

In [50]:
submission = test_A['id']

In [54]:
pred_class = pd.DataFrame(pred_class)

In [58]:
submission = pd.concat((submission,pred_class),axis=1)

C:\Users\dishant\AppData\Local\conda\conda\envs\my_root\lib\site-packages\pandas\core\indexes\api.py:77: RuntimeWarning: '<' not supported between instances of 'str' and 'int', sort order is undefined for incomparable objects
  result = result.union(other)


In [59]:
submission.head()

,id,0
0,0,1.0
1,1,1.0
2,2,1.0
3,3,1.0
4,4,1.0


In [60]:
submission.to_csv('submission1.csv',index=False)